In [ ]:
import pandas as pd
import configparser

In [236]:
path = "../data/data.h5"

config = configparser.ConfigParser()
config.read("config.ini")

def load_and_format(path):
    """
    Takes the path as arg and reads in the DataFrames from hdf-file.
    """
    output_df = dict()
    for name in config.sections():
        if name in ["rf_pv_Pod", "rf_gv_Pod", "rf_pv_Basis", "rf_gv_Basis"]:
            output_df[name] = (pd.read_hdf(path, key=name)
                                .pipe(extract_col)
                                .rename(columns=str.lower)
                                .drop([f"mode_costs_{i}" for i in range(1,7)], axis=1)
                                .assign(mode_1=lambda x: pd.Categorical(x["mode_1"]),
                                    mode_2=lambda x: pd.Categorical(x["mode_2"]),
                                    mode_3=lambda x: pd.Categorical(x["mode_3"]),
                                    mode_4=lambda x: pd.Categorical(x["mode_4"]),
                                    mode_5=lambda x: pd.Categorical(x["mode_5"]),
                                    mode_6=lambda x: pd.Categorical(x["mode_6"]),
                                    mode_length_1=lambda x: pd.to_numeric(x["mode_length_1"]),
                                    mode_length_2=lambda x: pd.to_numeric(x["mode_length_2"]),
                                    mode_length_3=lambda x: pd.to_numeric(x["mode_length_3"]),
                                    mode_length_4=lambda x: pd.to_numeric(x["mode_length_4"]),
                                    mode_length_5=lambda x: pd.to_numeric(x["mode_length_5"]),
                                    mode_length_6=lambda x: pd.to_numeric(x["mode_length_6"]),
                                    strecken_typ=lambda x: pd.Categorical(x["strecken_typ"])))
        elif name in ["Technologiedaten", "Verbesserungsfaktoren"]:
            output_df[name] = (pd.read_hdf(path, key=name)
                                .rename(columns=str.lower, index=str.lower))
        elif name == "Staedte":
            output_df[name] = (pd.read_hdf(path, key=name)
                                .rename(columns=str.lower)
                                .assign(stadt_typ=lambda x: pd.Categorical(x["stadt_typ"])))
        else:
            output_df[name] = pd.read_hdf(path, key=name)
    return output_df

# Use Funtion to load in DataFrames
#dfs= load_and_format(path)

def extract_col(df):
    """
    drops multiindex
    """
    df = df.copy()
    df.columns = df.columns.droplevel(0)
    return df


def load_dfs(path):
    dfs = dict()
    for name in config.sections():
        dfs[name] = pd.read_hdf(path, key=name)
    return dfs
        
dfs = load_dfs(path)
        
def save_dfs(path, dfs):
    for name in config.sections():
        dfs[name].to_hdf(path, format="table", key=name)
        
        
#save_dfs(path, dfs)

# Extract the DataFrames
cities = dfs["Staedte"]
tech_data = dfs["Technologiedaten"]
improve_data = dfs["Verbesserungsfaktoren"]
rf_pv_basis = dfs["rf_pv_Basis"]
rf_pv_pod = dfs["rf_pv_Pod"]
rf_gv_basis = dfs["rf_gv_Basis"]
rf_gv_pod = dfs["rf_gv_Pod"]
rf_pv_routes = dfs["rf_pv_Strecken"]
rf_gv_routes = dfs["rf_gv_Strecken"]

['config.ini']

In [261]:
rf_pv_basis.loc[:, ["mode_2", "mode_1"]]

,mode_2,mode_1
index,,
0.0,NaN,zu Fuss
1.0,NaN,Fahrrad
2.0,MIV,zu Fuss
3.0,U-Bahn,zu Fuss
4.0,NaN,E-Bike
...,...,...
216.0,Bus,zu Fuss
217.0,zu Fuss,Taxi
218.0,zu Fuss,Taxi


In [278]:
# Geschwindigkeiten berechnen

def calc_time_per_vehicle_and_row(row, av_speed, mode):
    for i in range(1,7):
        if row[f"mode_{i}"] == mode:
            row[f"mode_time_{i}"] = row[f"mode_length_{i}"]/av_speed
    return row

def av_calc(df_start, mode):
    df = df_start.copy()
    av_length = df.loc[
        (df.loc[:,"mode_1"] == mode)
        & (df.loc[:, "strecken_typ"] == "UK")
        & (df.loc[:, "mode_2"].isnull())
        ]["mode_length_1"].mean()
    av_time = df.loc[
        (df.loc[:,"mode_1"] == mode)
        & (df.loc[:, "strecken_typ"] == "UK")
        & (df.loc[:, "mode_2"].isnull())
        ]["transferzeit"].mean()
    av_speed = av_length/av_time
    print(mode)
    
    df = (df
        .apply(lambda x: calc_time_per_vehicle_and_row(x, av_speed, mode), axis = 1))
    
    print(f"Average Speed {mode}: {av_length/av_time} m/min")
    
    return df
    
    
for mode in ["E-Bike", "zu Fuss", "Fahrrad"]:
    rf_pv_basis = av_calc(rf_pv_basis, mode)
    
rf_pv_basis.loc[:,["mode_1", "mode_time_1"]]

E-Bike
Average Speed E-Bike: 366.6669542485916 m/min
zu Fuss
Average Speed zu Fuss: 79.87012987012987 m/min
Fahrrad
Average Speed Fahrrad: 268.42105263157896 m/min


,mode_1,mode_time_1
index,,
0.0,zu Fuss,31.300813
1.0,Fahrrad,9.686275
2.0,zu Fuss,5.008130
3.0,zu Fuss,6.260163
4.0,E-Bike,7.090904
...,...,...
216.0,zu Fuss,34.931707
217.0,Taxi,0.000000
218.0,Taxi,0.000000
